In [ ]:
from labdata.schema import *
import pylab as plt
from ipywidgets import IntSlider, interact
from decord import VideoReader
savepath = Path('../../figures/figure5')
savepath.mkdir(parents=True,exist_ok=True)

%matplotlib widget


In [ ]:
# time after implant for JC164 behavior recording in figure 5.

In [ ]:
key = dict(subject_name = 'JC164',session_name = '20240421_172012',
              video_name = 'top_view')
data = pd.DataFrame((PoseEstimation() & key).fetch())

fig = plt.figure()
ll = ['head','shoulder_right','shoulder_left','tail']
idx = data['label_name'].values == ll[0]
for l in ll[1:]:
    idx |= data['label_name'].values == l
    
ii = np.vstack(data[idx]['likelihood'].values) > 0.99
x = np.vstack(data[idx]['x'].values)
y = np.vstack(data[idx]['y'].values)
x[~ii] = np.nan
y[~ii] = np.nan
x = np.nanmean(x,axis = 0)
y = np.nanmean(y,axis = 0)
plt.plot(x,y,lw = 0.1,color = 'k')

key = dict(subject_name = 'JC164',session_name = '20240510_164038',
              video_name = 'top_view')
data = pd.DataFrame((PoseEstimation() & key).fetch())
import pylab as plt
%matplotlib widget
# plt.figure()
ll = ['head','shoulder_right','shoulder_left','tail']
idx = data['label_name'].values == ll[0]
for l in ll[1:]:
    idx |= data['label_name'].values == l
    
ii = np.vstack(data[idx]['likelihood'].values) > 0.99
x = np.vstack(data[idx]['x'].values)
y = np.vstack(data[idx]['y'].values)
x[~ii] = np.nan
y[~ii] = np.nan
x = np.nanmean(x,axis = 0)
y = np.nanmean(y,axis = 0)
plt.plot(x,y,lw = 0.1,color = 'r',alpha = 1)

pos = [[data[data['label_name'].values == 'bottom_left'].x.values[0].mean(),
        data[data['label_name'].values == 'bottom_left'].y.values[0].mean()],
       [data[data['label_name'].values == 'bottom_right'].x.values[0].mean(),
        data[data['label_name'].values == 'bottom_right'].y.values[0].mean()],
       [data[data['label_name'].values == 'top_right'].x.values[0].mean(),
        data[data['label_name'].values == 'top_right'].y.values[0].mean()],
       [data[data['label_name'].values == 'top_left'].x.values[0].mean(),
        data[data['label_name'].values == 'top_left'].y.values[0].mean()]]
pos = np.stack(pos)
plt.axis([np.min(pos[:,0]),np.max(pos[:,0]),np.min(pos[:,1]),np.max(pos[:,1])])

# plt.plot(*np.stack(pos).T,'k')
plt.plot([300,300+(20*np.max(pos[:,0])-np.min(pos[:,0]))/50],[200,200])
plt.title('JC164 mouse with 4 NP2 probes and 2 headstages')
fig.savefig(savepath/f'mouse4probe.pdf')

## Behavior and recording for the falling mouse

In [ ]:
key = dict(subject_name = 'JC131',
          session_name = '20231025_194303')

data = pd.DataFrame((PoseEstimation() & key).fetch())
dlc_labeling = {}
for bpart in data.label_name.values:
    dlc_labeling[bpart] = np.zeros((len(data[data.label_name == bpart]['x']),2))
    d = data[data.label_name == bpart].iloc[0]
    ii = (d['likelihood']<0.5)
    dlc_labeling[bpart] = np.vstack([d['x'],d['y']]).T
    dlc_labeling[bpart][ii] = np.nan
avg_pos = np.stack([dlc_labeling[p] for p in dlc_labeling.keys()])
avg_pos = np.nanmean(avg_pos,axis=0)
# read a frame
import cv2
cap = cv2.VideoCapture(str(((File() & (DatasetVideo.File() & key))).get()[0]))
cap.set(cv2.CAP_PROP_POS_FRAMES, 42900);
ret,frame = cap.read()
del cap

%matplotlib widget
fig = plt.figure(dpi = 100) # 600 for the fig
plt.imshow(frame[:,:,0],clim=[4,255],cmap='gray')
x,y =  (avg_pos[40000:,0],avg_pos[40000:,1])
plt.scatter(x,y,0.2,np.linspace(0.5,1,len(x)),alpha=0.8,
            cmap='jet',
            clim = [0,0.5],
            rasterized=True)
plt.axis('off')
# approximate scalebar (not corrected for prespective)
pix2cm = 4.5/(663-470)
plt.plot(np.array([1,6])/pix2cm,frame.shape[0]+np.array([-20,-20]),'k')
plt.text(3.5/pix2cm,frame.shape[0]-20,'~5 cm',va = 'bottom',ha='center')
plt.xlim([0,920])
plt.ylim([1020,200])

# figurespath = Path('../figures/freely_moving/')
# figurespath.mkdir(exist_ok=True)
fig.savefig(savepath/f'mouse_fall.pdf')


### Example dredge freely moving

In [ ]:
# Get the syncs and align the frames to the rasters
key = dict(subject_name = 'JC131',
          session_name = '20231025_194303')
ch_sma = 7
key = (Dataset() & (EphysRecording() & key)).proj().fetch1()
ephys = (DatasetEvents.Digital() & key & 'stream_name = "imec0"').fetch(as_dict = True)
srate = float((EphysRecording.ProbeSetting() & key & 'probe_num = 0').fetch1('sampling_rate'))
if not len(ephys):
    raise OSError('Could not find sync for probe 0 - check session.')
    
nidqfiles = (File() & (Dataset.DataFiles() & key & 'file_path LIKE "%.nidq.%"')).get()
for f in nidqfiles:
    if str(f).endswith('.bin'):
        from spks.spikeglx_utils import load_spikeglx_binary
        dat,meta = load_spikeglx_binary(f)
        from spks.sync import unpackbits_gpu
        onsets,offsets = unpackbits_gpu(dat[:,-1])
        
from spks.sync import interp1d
ni_ap_interp = interp1d(onsets[ch_sma],ephys[0]['event_values'],fill_value='extrapolate')
framesamples = ni_ap_interp(onsets[6]).astype('uint64')
# extract the encoder traces this takes a while, could probably be made f
frame_time = framesamples/srate

fallframe = 42297
falltime = frame_time[fallframe]
fallsample = framesamples[fallframe]

from scipy.interpolate import interp1d
from scipy.signal import savgol_filter
# remove nan from position and covert to pixels
z = interp1d(frame_time[np.isfinite(avg_pos[:,1])],  avg_pos[np.isfinite(avg_pos[:,1]),1],fill_value='extrapolate')(frame_time)
z = -1*(z-np.max(z))*pix2cm

In [ ]:
parameter_set = 5
shank = 3
from labdata import chronic_paper as paper
dredgedata = (paper.DredgeMotionEstimate() & dict(key,shank_num = shank)).fetch1()
dredge_spks = (paper.DredgeSpikeDetection() & key).get_spikes(shank_num = shank)
dredge_spks = dredge_spks.iloc[0]

key = (SpikeSorting & dict(key,parameter_set_num = parameter_set)).proj().fetch1()

sp,amps,depth = (SpikeSorting.Unit()*UnitMetrics()*UnitCount.Unit() & key & f'shank = {shank}' & 'passes = 1' & 'unit_criteria_id = 1').fetch('spike_times','spike_amplitudes','spike_positions')
nunits = len(sp)
sp = np.hstack(sp)/srate
amps = np.hstack([0*a + np.random.randint(len(amps)) for i,a in enumerate(amps)])
depth = np.vstack(depth)[:,1]

In [ ]:
fig = plt.figure(figsize = [5,4])
ax = fig.add_axes([0.1,0.1,0.8,0.6])
shank = 3
offset = 460

cmaplist = [cmap(i) for i in range(plt.cm.Set1.N)]
cmaplist[-1] = cmaplist[0]
from pylab import matplotlib as mpl
cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom', cmaplist, cmap.N)

from spks.viz import plot_drift_raster

idx = dredge_spks.t_seconds>offset
tidx = sp>offset
plot_drift_raster(dredge_spks.t_seconds[idx],dredge_spks.depth_um[idx],dredge_spks.amps[idx],cmap = 'gray_r',clim = [0,500])
plot_drift_raster(sp[tidx],
                  depth[tidx],
                  amps[tidx],n_spikes_to_plot=45000,cmap=cmap,markersize = 0.05,alpha = 1)
plt.xlabel(f'{nunits} single units (shank {shank})',loc = 'right',labelpad = 1,fontsize = 12)
plt.gca().xaxis.set_label_position('top') 

plt.plot(offset+np.array([0,120]),np.min(depth)-np.array([50,50]),clip_on=False,color='k')
plt.text(offset+60,np.min(depth)-60,'2 min',va = 'top',ha = 'center',fontsize = 12)
plt.xticks([]),plt.yticks([])
plt.plot(offset+np.array([-10,-10]),np.min(depth)+np.array([100,300]),clip_on=False,color='k')
plt.text(offset-10,np.min(depth)+200,'200$\mu$m',va = 'center',ha = 'right',rotation = 90,fontsize = 12);

a2 = fig.add_axes([0.1,0.8,0.8,0.15],sharex = ax)
i = frame_time>offset
plt.plot(frame_time[i], z[i],'k')
plt.xlim(offset,np.max(frame_time[i]))

plt.vlines(frame_time[fallframe],0,15,'r')
plt.yticks([0,15])
plt.ylabel('[cm]')
plt.text(offset+10,10,'Approx mouse z position')
# fig.savefig(figurespath/'freely_moving.png')
# fig.savefig(figurespath/'freely_moving.pdf')
plt.plot(dredgedata['time_bin_centers_s'],dredgedata['displacement'][0]+np.mean(plt.ylim()),'c')

fig.savefig(savepath/f'mouse_fall_dredge.pdf')


### Example implant for 4 probe freely moving

In [ ]:
import numpy as np
from vvasp.VizClasses import NeuropixelsChronicHolder
from vvasp.atlas_utils import Atlas as VvaspAtlas
import pyvista as pv
from labdata.schema import *
from labdata import chronic_paper as cp
import cv2
from pathlib import Path
from pyvistaqt import BackgroundPlotter
from tqdm import tqdm
plotter = BackgroundPlotter()

for subj2plot in ['JC164']:
    insertions = ProbeInsertion * cp.ChronicInsertion() & f'subject_name = "{subj2plot}"'
    insertions = insertions.fetch(as_dict=True)

    regions_to_show = cp.ChronicInsertion().TargetedRegion() & f'subject_name = "{subj2plot}"'
    regions_to_show = regions_to_show.fetch('region_acronym','hemisphere', as_dict=True)
    atlas = VvaspAtlas(plotter, min_tree_depth=6, max_tree_depth=8)
    for region in regions_to_show:
        atlas.add_atlas_region_mesh(region['region_acronym'], region['hemisphere'], force_replot=True)

    prbs = []
    for i, entrypoint in enumerate(insertions):
        entry_point = [entrypoint['insertion_ml'], entrypoint['insertion_ap']]
        depth = entrypoint['insertion_depth']
        angles = [entrypoint['insertion_el'], entrypoint['insertion_spin'], entrypoint['insertion_az']]
        if entrypoint['holder_id'] == 0:
            prb = NeuropixelsChronicHolder('NP1','head_fixed', plotter, root_intersection_mesh=atlas.meshes['root'])
        elif entrypoint['holder_id'] == 1:
            prb = NeuropixelsChronicHolder('NP1','freely_moving', plotter, root_intersection_mesh=atlas.meshes['root'])
        elif entrypoint['holder_id'] == 2:
            prb = NeuropixelsChronicHolder('NP24','head_fixed', plotter, root_intersection_mesh=atlas.meshes['root'])
        elif entrypoint['holder_id'] == 3:
            prb = NeuropixelsChronicHolder('NP24','freely_moving', plotter, root_intersection_mesh=atlas.meshes['root'])
        elif entrypoint['holder_id'] == 4:
            prb = NeuropixelsChronicHolder('NP24a','head_fixed', plotter, root_intersection_mesh=atlas.meshes['root'])
        elif entrypoint['holder_id'] == 5:
            prb = NeuropixelsChronicHolder('NP24a','freely_moving', plotter, root_intersection_mesh=atlas.meshes['root'])
        else:
            raise ValueError('Holder ID not recognized')

        prb.drive_probe_from_entry(entry_point, angles, depth)
        prb.make_inactive()
        prbs.append(prb)

    plotter.isometric_view()

In [ ]:
img = plotter.screenshot(savepath / f'{subj2plot}_trajectories.png', return_img=True, scale=5)